# $$
\text{African Institute for Mathematical Sciences, Rwanda}
$$

## $$
\text{Algebra and Cryptography Assignment 1: Code and Algorithms Part}
$$

### $$
\text{Chukwuezi Tochukwu Precious}
$$

$$
\text{March 1, 2025}
$$

# $\text{Solutions 1: Inverse Functions of Cyclic Groups}$

For ease and accuracy in computations, I made this fuction to help me in question Exercise 1.1

It solves the inverse of $2$ and $3$ in 

1. $(2^{-1})^2 \mod{13}$
2. $(3^{-1})^2 \mod{13}$

In [279]:
# Inverse function for Exercise 1.3

def a_inverse(a, n):
    i = xgcd(n, a)[-1]
    i = (i + n)%n
    return i


print(a_inverse(2, 13))
print(a_inverse(3, 13))

7
9


# $\text{Solutions 2: Implementation of RSA Cryptosystem}$

To implement an encrypt-decrypt system as described in the question below.

Bob intercepts from Alice the following encrypted message:

$$
[427849968240759007228494978639775081809\\
498308250136673589542748543030806629941\\
925288105342943743271024837479707225255]
$$

Alice knows that Bob uses RSA cryptosystem and his public key is $(12398737; n)$ where
$$n = 956331992007843552652604425031376690367$$

Knowing that Alice and Bob agreed to use RSA cryptosystem to communicate in secret, each
message consist of a single letter which is encoded as: $\text{Space = 00;A = 11;B = 12; ...;Z =
36;}$ which message did Alice sent to Bob?

### $2.0 \quad \text{Useful First Steps}$

Firstly, we begin by implementing the useful functions, one of which is the Euler's Totient function.

In [280]:
def euler_totient_sage(N):
    """
    Computes phi(N) using SageMath's optimized factorization.
    """
    if N < 1:
        print("n must be positive!")
        return
    if N == 1:
        return 1
    result = N
    for p in prime_divisors(N):
        result = result * (p - 1) // p
    return result

Next, we define n as given in the problem.

In [184]:
n = 956331992007843552652604425031376690367

### $2.1 \quad \text{Computing }\Phi(n).$

We now compute $\Phi(n)$, the Euler's totient function on $n$, that gives the order of the cycle $\bigg(\frac{\mathbb{Z}}{n\mathbb{Z}}\bigg)$

In [185]:
phi = euler_totient_sage(n)
phi

956331992007843552521401346814050873280

### $2.2 \quad \text{Verifying the Exponent of the RSA}$

In [247]:
e, n = (12398737, 956331992007843552652604425031376690367)

# We then call the inbuilt xgcd function that returns gcd, u and v for au + bv = gcd(a, b)
d = xgcd(e, phi)[0]
d

1

Since the $\gcd(e, \phi(n)) = 1$, the exponent $e$ is a valid exponent of the $RSA$ system.

### $2.3 \quad \text{Decipherig the Secret Key used to Encrypt the Message}$

In [248]:
sk = xgcd(phi, e)[2]
sk

-154575730004375681679140546242099203407

In [249]:
sk += phi #This is because sk is was negative.
sk

801756262003467870842260800571951669873

Having obtained the secret key ($d$ in theory, $sk$ in `code`), we can ensure it satisfies that,
$$
de \equiv 1 \mod \Phi(n)
$$

In [187]:
(sk * e)%phi

1

And thus, we have obtained our secret key. We also observe something as shown below. Our secret key has similar length with our $n$.

In [191]:
len(str(sk)) == len(str(n))

True

### $2.4 \quad \text{Decrypting the Encrypted Message}$

Given the encrypted message, we want to see what it holds now we know the public key and the secret key.

In [190]:
M_encrypt = [427849968240759007228494978639775081809,
498308250136673589542748543030806629941,
925288105342943743271024837479707225255,
95024328800414254907217356783906225740]

print(M_encrypt)

[427849968240759007228494978639775081809, 498308250136673589542748543030806629941, 925288105342943743271024837479707225255, 95024328800414254907217356783906225740]


The next thing we do is to decrypt each chunk of the message, by using the formula:
$$
M = m^{d} \mod n
$$
Where $m$ is an encrpyted chunk, and $d$ is the secret key.

In [192]:
M_dcrypt = [power_mod(m, sk, n) for m in EM]

print(M_dcrypt)

[30181929001929002335002215303015280030, 25003018150033252822140030181130002415, 32152800332825301500302500231500152319, 223500141913211924292524]


We can check the lengths of each decrypted message chunk.

In [193]:
[len(str(e)) for e in M_dcrypt]

[38, 38, 38, 24]

### $2.5 \quad \text{Implementing the Dictionary that Decodes The Message}$

Two powerful `python` implementations to encode and decode letters is the `ord` and the `chr` functions used to convert from letters to numbers and vice versa respectively.

In [205]:
# Given that orb("A") = 65 and ord("Z") = 90, we implement the algorithm as follows, with shift from [11, 36] to [65, 90]
spaces = {"00" : " "}

encode_logic = {f"{k}" : chr(k + 65 - 11) for k in range(11, 37)}

encode_logic.update(spaces)

print(encode_logic)

{'11': 'A', '12': 'B', '13': 'C', '14': 'D', '15': 'E', '16': 'F', '17': 'G', '18': 'H', '19': 'I', '20': 'J', '21': 'K', '22': 'L', '23': 'M', '24': 'N', '25': 'O', '26': 'P', '27': 'Q', '28': 'R', '29': 'S', '30': 'T', '31': 'U', '32': 'V', '33': 'W', '34': 'X', '35': 'Y', '36': 'Z', '00': ' '}


Next, we will convert each chunk of the decrpyted message to strings for easy handling.

In [206]:
M_as_str = [str(I) for I in DM] # I is the large integer number representing the decrypted message.
print(M_as_str)

['30181929001929002335002215303015280030', '25003018150033252822140030181130002415', '32152800332825301500302500231500152319', '223500141913211924292524']


In [ ]:
We can now finally define a logic to translate the message and return the final messag

In [207]:
def translate(message_array, dictionary_logic):
    """
    This function receives parameters help translate a decrypted message string to the decoded string
    using the dictionay encoding logic.
    
    Input: 
        -message_array : String to translate,
        -dictionary_logic : The dictionary that houses the logic to decode the numbers (in pairs) in our case
        
    Output:
        -a text containing the translated message.
    """
    text = ""
    
    # Group numbers in two and translate
    for message in message_array:
        for i in range(0, len(message), 2):
            char_key = message[i] + message[i+1] # get the character key for each pair.
            text += dictionary_logic[char_key]
    return text

In [217]:
%%time
print(F"Deciphered: {translate(M_as_str, encode_logic)}")
print("\n\nTime Taken:\n")

Deciphered: THIS IS MY LETTER TO THE WORLD THAT NEVER WROTE TO ME EMILY DICKINSON


Time Taken:

CPU times: user 513 µs, sys: 0 ns, total: 513 µs
Wall time: 458 µs


What a poetic message to encrypt!

# $\text{Solution 3: Building My Own RSA System}$

Create your own public key and private key for $RSA$ cryptosystem. The two prime numbers
must have $600$ digits and they have to be safe prime numbers. Then, Set up your own RSA
cryptosystem. Demonstrate how a message addressed to you can be encrypted and how you
can decrypt it using your private key.

### $3.0 \quad \text{Implementing All The Useful Algorithms}$

Here, I implement and design some modules/functions to check primality based on Algorithm $3.7$ in the Cryptoschool book used in class.

In [218]:
import random
shuffle = random.SystemRandom()

# This fuction uses the Rabin-Miller test to check the primeness of a number just once.
# It follows the Algorithm discussed in 3.7, but does the test for only one witness per time. 
# It uses bitwise operations to speed things up.

def is_singly_prime(n, x):
    """
    Determines if n passes the Miller-Rabin primality test for base x.
    
    Input: n (number to test), x (base)
    Output: True if n passes, False otherwise
    """
    e = n - 1
    while not e & 1:
        e >>= 1  # Divide by 2 until e is odd
    if power_mod(x, e, n) == 1:
        return True
    while e < n - 1:
        if power_mod(x, e, n) == n - 1:
            return True
        e <<= 1  # Multiply by 2 (equivalent to squaring)
    return False


# This is the main implementation of of the Miller-Rabin test for primality.
# A stochastic approach

def pseudoprimality_of(n, t=20):
    """
    Performs t iterations of the Miller-Rabin test with random bases.
    
    Input: n (number to test), t (number of trials)
    Output: False if composite is detected, True otherwise
    """
    i = 1
    while i <= t:
        x = shuffle.randrange(2, n - 1) # witnesses
        if not is_singly_prime(n, x):
            return False
        i += 1
    return True


# The safe prime algorithm uses tecniques discussed in class and the safe prime algorithm discussed in 
# 3.24 to ensure in addition the pseudoprimality test algorithm, the prime number returned is safe.

def is_safe_prime(p):
    """
    Given a prime number p, this function tests if the prime number satisfies p = 2l + 1, where l is prime.
    In other words, if (p - 1)/2 is prime, or in most cases, a product of primes (around 2), then p is safe.
    """
    l = (p - 1) >> 1  # Compute l = (p-1)/2
    return pseudoprimality_of(p, 10) and pseudoprimality_of(l, 10)


# The generation of prime numbers uses the Rabin Miller Pseudoprimality test in conjunction with the safe prime number test
# to generate a large prime number with n_bits of base 2 digits which is <= 2^(nbits + 1) - 1

    
def generate_large_safe_prime(n_bits):
    """
    Generates a probable prime with n_bits+1 bits (due to original code's logic).
    
    Input: n_bits (target bit-length minus one)
    Output: A probable prime number
    """
    while True:
        # Generates an odd number in the range [2^(n_bits)+1, 2^(n_bits+1)-1]
        primeN = (shuffle.randrange(1 << (n_bits - 1), 1 << n_bits) << 1) + 1
        
        if is_safe_prime(primeN):
            return primeN

# Finally, I implemented a functionality to display the prime numbers gotten in a more presentable way.

def display_prime(*any):
    """This Function Helps us display our safe prime number"""
    
    for prime in any:
        print(f"The safe prime number is:\n")
        print(prime)
        print("\t")
        print(f"\nIt's length is {len(str(prime))}")

### $3.1 \quad \text{Generating The Two Prime Numbers, the Public Key (e, N) and the Secret Key (d)}$

The two prime numbers $pq = N$ representing the public key $(e, N)$ each have $600$ decimal digits, equivalent to $1992$ base $2$ bits.

In general, the public key is of the form:
$
(e, N)
$

Where $e$ is the exponent and $N$ the public key parameter is analogous to $n$ previously encountered in Solution $2$.

### $3.2 \quad \text{Generating $p$, $q$ and $N$}$

To we first generate the primes p and q with the algorithms above.

In [161]:
%%time
p = generate_large_safe_prime(1992)
display_prime(p)

The prime number is:

493644220098836803634031028467830307425986257960539824377485078506203132829751630881054548618868101722879012609267713585555527703110921872160464014698569921623442519850559468151131795633281160978780925697641569511910320276829043685429912228406379593885901280607216010970016889104930876539623100734421686775486198987435851068031214360512997620566113643578220695779320331455521450613746912907252315995230386288353945499836210924564631851878183158719594301775691467961086047507318074162793934485878305681400220922871677234626363869042613080265650142804541634884410144816245683722033771757628103239932787
	

It's length is 600
CPU times: user 19min 43s, sys: 517 ms, total: 19min 44s
Wall time: 19min 44s


In [225]:
%%time

is_prime(p)

CPU times: user 23.1 s, sys: 50.1 ms, total: 23.2 s
Wall time: 23 s


True

In [163]:
%%time
q = generate_large_safe_prime(1992)
display_prime(q)

The prime number is:

534085579822278606703655558848960547656496842925378871617229425348509158806695066049058310481882206654866152722110593117898402285264397964991038543246345577760302623712365488581429835047923019779974620298287663455478598435384026103996382245993865960292911587827358451485005377034211877660484293701329811715046469345136682901986523554204569990365041753372631510726647850737259149253379953735346023559208243971783831393030897165997831191041268175847768635484717621394967486701448646396560457167921612916191324483099292665474911147814434525283358281145542841091585857142755518185934842581951308368215187
	

It's length is 600
CPU times: user 1h 14min 39s, sys: 2.05 s, total: 1h 14min 41s
Wall time: 1h 14min 41s


In [226]:
%%time

is_prime(q)

CPU times: user 33.4 s, sys: 190 ms, total: 33.6 s
Wall time: 33.2 s


True

Having generated the public and private keys, and have checked that they are both primes (safe prime, after passing the tests in the aloorithms), we can now define $N = pq$.

In [228]:
%%time
N = p*q #Public Key

N

CPU times: user 64 µs, sys: 0 ns, total: 64 µs
Wall time: 95.1 µs


2636482595174037730055749746230996945192429812896605102248665582330172921969796687696990372698070706037116135636514908308154721595008104314817922812481820521621038669729482866288708651301796295905637207812718322387622740991241927758585328742980221805860089868103968613887647498194856972399691620713491939937754007184519040088729527318547491406123314947099505870297367810957882503189359788011487338195761608140718172187077143781846238699588002891797902765429036072886330032108207573803007933463552206752871294873939163599108562851994809578762749166223298642425619170286570364310489206610993468018667698593627463664477599945733849197722593131659931434020893913983105271682493892414867149886707675602280226198751953650548303428367728668573672359404033556605958654059168750477950350225785543417427836277564098232397106388303385284383699785791627145851082176379362156645097292080707099706475319909223800446233404451825367591699486121164587682065062925488486539399476844885496006832554850373293020555882788

### $3.3 \quad \text{The Public Key}$

For the exponent of the public key, we choose $e = 5$, hence
$$
\text{Public key} : (5, N)
$$

Where, $$ N = pq$$

We have to also ensure that for $e = 5$,

$$
\gcd(e, \Phi(N)) = 1
$$

The demonstration is shown below.

In [255]:
%%time
phin = (p - 1)*(q - 1) #euler_totient_sage(N)
phin

CPU times: user 63 µs, sys: 0 ns, total: 63 µs
Wall time: 67.5 µs


2636482595174037730055749746230996945192429812896605102248665582330172921969796687696990372698070706037116135636514908308154721595008104314817922812481820521621038669729482866288708651301796295905637207812718322387622740991241927758585328742980221805860089868103968613887647498194856972399691620713491939937754007184519040088729527318547491406123314947099505870297367810957882503189359788011487338195761608140718172187077143781846238699588002891797902765429036072886330032108207573803007933463552206752871294873939163599108562851994809578762749166223298642425619170286570364310489206610993468018667688316329464453323496568867976029814042306828922574833933966838066724559577527947897848758116668099196448747098639867481268889067844915375300844378454107450964816607733121228383024609478731375620248722104138940067432499116263153685805522846883143395540388250677810900472741858045708278933318835279442931248499125142041866359785943785440506388953613934517030877411785203674079026556179104626594572487244

In [256]:
# We want to show that the exponent 5 is an exponent of the RSA we are building.
e = 5
gcd  = xgcd(e, phin)[0]
gcd

1

Hence $e = 5$ forms a good choice for the exponent of our $RSA$ since $\gcd(5, \Phi(N)) = 1$.

### $3.4 \quad \text{The Secret Key}$

 We can solve for d (secret key) by finding the value of v satisfying:
 $$
 \Phi(N) u + ev = 1
 $$

In [262]:
d_sk =  xgcd(phin, e)[2]
d_sk += phin
d_sk

2109186076139230184044599796984797556153943850317284081798932465864138337575837350157592298158456564829692908509211926646523777276006483451854338249985456417296830935783586293030966921041437036724509766250174657910098192792993542206868262994384177444688071894483174891110117998555885577919753296570793551950203205747615232070983621854837993124898651957679604696237894248766306002551487830409189870556609286512574537749661715025476990959670402313438322212343228858309064025686566059042406346770841765402297035899151330879286850281595847663010199332978638913940495336229256291448391365288794774414934150653063571562658797255094380823851233845463138059867147173470453379647662022358318279006493334479357158997678911893985015111254275932300240675502763285960771853286186496982706419687582985100496198977683311152053945999293010522948644418277506514716432310600542248720378193486436566623146655068223554344998799300113633493087828755028352405111162891147613624701929428162939263221244943283701275657989795

if d is the private key, then it should satisfy

$$
ed \equiv 1 \mod \Phi(p)
$$

Evaluating this value with $e = 5$,

In [263]:
power_mod(e*d_sk, 1, phin)

1

### $3.5 \quad \text{The Encryption Function}$

We define our encription function as

\begin{align*}
f : &M \rightarrow M\\
&m \mapsto m^{e} \mod N
\end{align*}

So let $m = 15678822656$ be addressed to me. It's encrypted version is $f(m)$, given in the code below

In [272]:
def f(m):
    """
    Encrypts data using:
    
    Input: m (message to encrypt)
    Defaults: e (exponent of RSA), N = public key parameter.
    
    Output: c (message encrypted)
    """
    return power_mod(m, e, N)

m = 15678822656
c = f(m)
c

947473864525952903487247265785497626718136514379776

This means that the encrypted message will be as shown above.

### $3.6 \quad \text{The Decryption Function}$

Given a secret key $d$, and $f(m) = c$, we define the decryption functionality as

\begin{align*}
f^{-1} : &M \rightarrow M\\
&c \mapsto c^{d} \mod N
\end{align*}

Where $g = f^{-1}(c)$.

So given that $f(m)$ was evaluated, we can now define the inverse fuction as follows:

In [274]:
def g(c):
    """
    Decrypts data using:
    
    Input: c (cyphered or encrypted message)
    Defaults: d (inverse of exponent of RSA, secret key), N = public key parameter.
    
    Output: c (message encrypted)
    """
    return power_mod(c, d_sk, N)

g(c)

15678822656

Which is exactly the value we chose for m. What a way to encrypt data. I only hope it's secured enough!

### $3.7\quad \text{Using my New Keys on the Exercise 2}$

In [282]:
encrypted_2 = list(map(lambda m : f(m), M_dcrypt))

encrypted_2

[25045803319645369664432632248425344220082801105181336957700808781923826707405998313417149257072705637274251939434825708370515872825501411285691736170684425082128906122282184379884024300000,
 9771521247772667305066514540542737080508947142199134475758860204568973023708845011060814459823271365184034817282657456540473316091105046081614496518686846897724562116178050204390258259375,
 34363233081756891694362885263365779757593402880768961921355033656654704148543821174718384609471833802096777009002717944119267474648520578792330690102665572512413809698622789684569053017599,
 557685067624806149506455903881717575803941402664282280529826728148382511176235805085043448294809002269852616350362624]

In [283]:
decrypted_2 = list(map(lambda c : g(c), encrypted_2))

decrypted_2

[30181929001929002335002215303015280030,
 25003018150033252822140030181130002415,
 32152800332825301500302500231500152319,
 223500141913211924292524]

In [284]:
%%time
print(F"Deciphered: {translate([str(m) for m in decrypted_2], encode_logic)}")
print("\n\nTime Taken:\n")

Deciphered: THIS IS MY LETTER TO THE WORLD THAT NEVER WROTE TO ME EMILY DICKINSON


Time Taken:

CPU times: user 8.9 ms, sys: 0 ns, total: 8.9 ms
Wall time: 8.7 ms
